## Loading in data

In [1]:
import pandas as pd
import numpy as np

from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers

INFO:rdflib:RDFLib Version: 4.2.1


### The code in the section is what I used to extract the metadata from the Project Gutenberg website. I would like to credit [Andreas van Cranenburgh](https://gist.github.com/andreasvc/b3b4189120d84dec8857) for making this code available on Github.

In [2]:
import os
import re
import gzip
import tarfile
import urllib
import xml.etree.cElementTree as ElementTree
try:
    import cPickle as pickle
except ImportError:
    import pickle


In [3]:
PICKLEFILE = '/tmp/md.pickle.gz'  # The Python dict produced by this module
RDFFILES = '/tmp/rdf-files.tar.bz2'  # The catalog downloaded from Gutenberg
RDFURL = r'http://www.gutenberg.org/cache/epub/feeds/rdf-files.tar.bz2'
META_FIELDS = ('id', 'author', 'title', 'downloads', 'formats', 'type', 'LCC',
		'subjects', 'authoryearofbirth', 'authoryearofdeath', 'language')
NS = dict(
		pg='http://www.gutenberg.org/2009/pgterms/',
		dc='http://purl.org/dc/terms/',
		dcam='http://purl.org/dc/dcam/',
		rdf='http://www.w3.org/1999/02/22-rdf-syntax-ns#')
LINEBREAKRE = re.compile(ur'[ \t]*[\n\r]+[ \t]*')
ETEXTRE = re.compile(r'''
	e(text|b?ook)
	\s*
	(\#\s*(?P<etextid_front>\d+)
	|
	(?P<etextid_back>\d+)\s*\#)
	''', re.IGNORECASE | re.VERBOSE)


def readmetadata():
	"""Read/create cached metadata dump of Gutenberg catalog.
	Returns:
		A dictionary with the following fields:
		id (int): Gutenberg identifier of text
		author (str): Last name, First name
		title (str): title of work
		subjects (list of str): list of descriptive subjects; a subject may be
			hierarchical, e.g:
			'England -- Social life and customs -- 19th century -- Fiction'
		LCC (list of str): a list of two letter Library of Congress
			Classifications, e.g., 'PS'
		language (list of str): list of two letter language codes.
		type (str): 'Text', 'Sound', ...
		formats (dict of str, str pairs): keys are MIME types, values are URLs.
		download count (int): the number of times this ebook has been
			downloaded from the Gutenberg site in the last 30 days.
	Fields that are not part of the metadata are set to None.
	http://www.gutenberg.org/wiki/Gutenberg:Help_on_Bibliographic_Record_Page
	"""
	if os.path.exists(PICKLEFILE):
		metadata = pickle.load(gzip.open(PICKLEFILE, 'rb'))
	else:
		metadata = {}
		for xml in getrdfdata():
			ebook = xml.find(r'{%(pg)s}ebook' % NS)
			if ebook is None:
				continue
			result = parsemetadata(ebook)
			if result is not None:
				metadata[result['id']] = result
		pickle.dump(metadata, gzip.open(PICKLEFILE, 'wb'), protocol=-1)
	return metadata

def getrdfdata():
	"""Downloads Project Gutenberg RDF catalog.
	Yields:
		xml.etree.ElementTree.Element: An etext meta-data definition.
	"""
	if not os.path.exists(RDFFILES):
		_, _ = urllib.urlretrieve(RDFURL, RDFFILES)
	with tarfile.open(RDFFILES) as archive:
		for tarinfo in archive:
			yield ElementTree.parse(archive.extractfile(tarinfo))

def parsemetadata(ebook):
	"""Parses an etext meta-data definition to extract fields.
	Args:
		ebook (xml.etree.ElementTree.Element): An ebook meta-data definition.
	"""
	result = dict.fromkeys(META_FIELDS)
	# get etext no
	about = ebook.get('{%(rdf)s}about' % NS)
	result['id'] = int(os.path.basename(about))
	# author
	creator = ebook.find('.//{%(dc)s}creator' % NS)
	if creator is not None:
		name = creator.find('.//{%(pg)s}name' % NS)
		if name is not None:
			result['author'] = safeunicode(name.text, encoding='utf-8')
		birth = creator.find('.//{%(pg)s}birthdate' % NS)
		if birth is not None:
			result['authoryearofbirth'] = int(birth.text)
		death = creator.find('.//{%(pg)s}deathdate' % NS)
		if death is not None:
			result['authoryearofdeath'] = int(death.text)
	# title
	title = ebook.find('.//{%(dc)s}title' % NS)
	if title is not None:
		result['title'] = fixsubtitles(
				safeunicode(title.text, encoding='utf-8'))
	# subject lists
	result['subjects'], result['LCC'] = set(), set()
	for subject in ebook.findall('.//{%(dc)s}subject' % NS):
		res = subject.find('.//{%(dcam)s}memberOf' % NS)
		if res is None:
			continue
		res = res.get('{%(rdf)s}resource' % NS)
		value = subject.find('.//{%(rdf)s}value' % NS).text
		if res == ('%(dc)sLCSH' % NS):
			result['subjects'].add(value)
		elif res == ('%(dc)sLCC' % NS):
			result['LCC'].add(value)
	# formats
	result['formats'] = {file.find('{%(dc)s}format//{%(rdf)s}value' % NS).text:
			file.get('{%(rdf)s}about' % NS)
			for file in ebook.findall('.//{%(pg)s}file' % NS)}
	# type
	booktype = ebook.find('.//{%(dc)s}type//{%(rdf)s}value' % NS)
	if booktype is not None:
		result['type'] = booktype.text
	# languages
	lang = ebook.findall('.//{%(dc)s}language//{%(rdf)s}value' % NS)
	result['language'] = [a.text for a in lang] or None
	# download count
	downloads = ebook.find('.//{%(pg)s}downloads' % NS)
	if downloads is not None:
		result['downloads'] = int(downloads.text)
	return result

def etextno(lines):
	"""Retrieves the id for an etext.
	Args:
		lines (iter): The lines of the etext to search.
	Returns:
		int: The id of the etext.
	Raises:
		ValueError: If no etext id was found.
	Examples:
		>>> etextno(['Release Date: March 17, 2004 [EBook #11609]'])
		11609
		>>> etextno(['Release Date: July, 2003 [Etext# 4263]'])
		4263
		>>> etextno(['Release Date: November 29, 2003 [Eook #10335]'])
		10335
		>>> etextno(['December, 1998  [Etext 1576#]'])
		1576
		>>> etextno(['Some lines', 'without', 'Any [Etext] Number'])
		Traceback (most recent call last):
			...
		ValueError: no etext-id found
	"""
	for line in lines:
		match = ETEXTRE.search(line)
		if match is not None:
			front_match = match.group('etextid_front')
			back_match = match.group('etextid_back')
			if front_match is not None:
				return int(front_match)
			elif back_match is not None:
				return int(back_match)
			else:
				raise ValueError('no regex match (this should never happen')
	raise ValueError('no etext-id found')


def fixsubtitles(title):
	"""Introduce any subtitle with (semi)colons instead of newlines.
	The first subtitle is introduced with a colon, the rest with semicolons.
	>>> fixsubtitles(u'First Across ...\r\nThe Story of ... \r\n'
	... 'Being an investigation into ...')
	u'First Across ...: The Story of ...; Being an investigation into ...'"""
	tmp = LINEBREAKRE.sub(': ', title, 1)
	return LINEBREAKRE.sub('; ', tmp)


def safeunicode(arg, *args, **kwargs):
	"""Coerce argument to unicode, if it's not already."""
	return arg if isinstance(arg, unicode) else unicode(arg, *args, **kwargs)

__all__ = ['readmetadata']

In [4]:
x = readmetadata()

In [11]:
# This is what the format of x looks like:
x[1]

{'LCC': {'E201', 'JK'},
 'author': u'Jefferson, Thomas',
 'authoryearofbirth': 1743,
 'authoryearofdeath': 1826,
 'downloads': 737,
 'formats': {'application/epub+zip': 'http://www.gutenberg.org/ebooks/1.epub.images',
  'application/prs.tex': 'http://www.gutenberg.org/6/5/2/6527/6527-t/6527-t.tex',
  'application/rdf+xml': 'http://www.gutenberg.org/ebooks/1.rdf',
  'application/x-mobipocket-ebook': 'http://www.gutenberg.org/ebooks/1.kindle.noimages',
  'application/zip': 'http://www.gutenberg.org/files/1/1.zip',
  'text/html': 'http://www.gutenberg.org/ebooks/1.html.noimages',
  'text/plain': 'http://www.gutenberg.org/ebooks/1.txt.utf-8',
  'text/plain; charset=us-ascii': 'http://www.gutenberg.org/files/1/1.txt'},
 'id': 1,
 'language': ['en'],
 'subjects': {'United States -- History -- Revolution, 1775-1783 -- Sources',
  'United States. Declaration of Independence'},
 'title': u'The Declaration of Independence of the United States of America',
 'type': 'Text'}

In [6]:
meta_df = pd.DataFrame(x)

In [13]:
meta_df = meta_df.T
meta_df.head(2)

,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,language,subjects,title,type
0,{},None,None,None,None,{},0,None,{},None,Text
1,"{E201, JK}","Jefferson, Thomas",1743,1826,737,{u'text/html': u'http://www.gutenberg.org/eboo...,1,[en],"{United States. Declaration of Independence, U...",The Declaration of Independence of the United ...,Text


In [15]:
meta_df.shape

(54124, 11)

## Now I have a dataframe that has metadata about all 54,124 books

### My next steps are to clean this up a little to make it useable. I will need to:
- Remove row 0 (there is no book with id 0)
- Remove anything that is not in a text format, such as audiobooks or datasets.
- Remove books that are not written in English
- Remove books to which access has been blocked (there are several books like this, probably because of copyright disputes)

In [16]:
meta_df.head(3)

,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,language,subjects,title,type
0,{},None,None,None,None,{},0,None,{},None,Text
1,"{E201, JK}","Jefferson, Thomas",1743,1826,737,{u'text/html': u'http://www.gutenberg.org/eboo...,1,[en],"{United States. Declaration of Independence, U...",The Declaration of Independence of the United ...,Text
2,"{KF, JK}",United States,None,None,212,{u'text/html': u'http://www.gutenberg.org/file...,2,[en],{United States. Constitution. 1st-10th Amendme...,The United States Bill of Rights: The Ten Orig...,Text


In [17]:
meta_df = meta_df.sort_values(by = 'id')

In [18]:
meta_df.reset_index(inplace = True, drop=True)
meta_df.head(4)

,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,language,subjects,title,type
0,{},None,None,None,None,{},0,None,{},None,Text
1,"{E201, JK}","Jefferson, Thomas",1743,1826,737,{u'text/html': u'http://www.gutenberg.org/eboo...,1,[en],"{United States. Declaration of Independence, U...",The Declaration of Independence of the United ...,Text
2,"{KF, JK}",United States,None,None,212,{u'text/html': u'http://www.gutenberg.org/file...,2,[en],{United States. Constitution. 1st-10th Amendme...,The United States Bill of Rights: The Ten Orig...,Text
3,{E838},"Kennedy, John F. (John Fitzgerald)",1917,1963,34,{u'text/html': u'http://www.gutenberg.org/file...,3,[en],{United States -- Foreign relations -- 1961-19...,John F. Kennedy's Inaugural Address,Text


In [19]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54124 entries, 0 to 54123
Data columns (total 11 columns):
LCC                  54124 non-null object
author               52143 non-null object
authoryearofbirth    39308 non-null object
authoryearofdeath    38381 non-null object
downloads            54122 non-null object
formats              54124 non-null object
id                   54124 non-null object
language             54122 non-null object
subjects             54124 non-null object
title                54049 non-null object
type                 54124 non-null object
dtypes: object(11)
memory usage: 4.5+ MB


## I only want to keep books with type = Text
I won't be able to do much with audiobooks or video. Interestingly, the 'Dataset' type includes data like the first 1 million digets of 1/pi and some human chromosomes. I would love to come back and so some analysis on these datasets in a future project!

In [20]:
meta_df['type'].unique()

array(['Text', 'Dataset', 'StillImage', 'MovingImage', 'Sound', 'Image',
       'Collection'], dtype=object)

In [22]:
meta_df = meta_df[meta_df['type'] == 'Text']

## Removing null weird row

In [23]:
meta_df.tail(2)

,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,language,subjects,title,type
54122,{},"Burgess, Gelett",1866,1951,0,{u'image/jpeg': u'http://www.gutenberg.org/cac...,54124,[en],{},War the Creator,Text
54123,{},None,None,None,None,{},999999,None,{},Piccole anime,Text


In [24]:
meta_df = meta_df[~ meta_df['language'].isnull()]

In [25]:
meta_df.tail(2)

,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,language,subjects,title,type
54121,{},"Potapenko, I. N.",None,None,0,{u'text/html': u'http://www.gutenberg.org/eboo...,54123,[fi],{},Nuoruuden tunnustuksia: Romaani,Text
54122,{},"Burgess, Gelett",1866,1951,0,{u'image/jpeg': u'http://www.gutenberg.org/cac...,54124,[en],{},War the Creator,Text


In [26]:
meta_df.reset_index(inplace = True)

## Cleaning the language column

I only want books that are exclusively written in English

In [27]:
type(meta_df['language'][0])

list

In [28]:
def clean_language(lang_list):
    if 'en' in lang_list and len(lang_list) == 1:
        return 1
    else:
        return 0
    
meta_df['english'] = meta_df['language'].map(clean_language)
meta_df.head(2)

,index,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,language,subjects,title,type,english
0,1,"{E201, JK}","Jefferson, Thomas",1743,1826,737,{u'text/html': u'http://www.gutenberg.org/eboo...,1,[en],"{United States. Declaration of Independence, U...",The Declaration of Independence of the United ...,Text,1
1,2,"{KF, JK}",United States,None,None,212,{u'text/html': u'http://www.gutenberg.org/file...,2,[en],{United States. Constitution. 1st-10th Amendme...,The United States Bill of Rights: The Ten Orig...,Text,1


In [29]:
meta_df = meta_df[meta_df['english'] == 1]

In [30]:
meta_df.shape

(43066, 13)

## Now I'm going to start dealing with the subjects

In [31]:
meta_df['subjects'].head()

#here's what they look like

0    {United States. Declaration of Independence, U...
1    {United States. Constitution. 1st-10th Amendme...
2    {United States -- Foreign relations -- 1961-19...
3    {Consecration of cemeteries -- Pennsylvania --...
4    {United States. Constitution, United States --...
Name: subjects, dtype: object

In [32]:
type(meta_df.subjects[0])

set

In [33]:
meta_df.subjects[0]

{'United States -- History -- Revolution, 1775-1783 -- Sources',
 'United States. Declaration of Independence'}

In [34]:
import string

def subject_cleaning(subj_set):
    subjs = []
    for s in subj_set:
        s = s.split('--')
        s = [string.strip(i) for i in s]
        subjs.extend(s)
    return np.unique(subjs) 
        
meta_df['subjects2'] = meta_df.subjects.map(subject_cleaning)
meta_df.head(3)

,index,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,language,subjects,title,type,english,subjects2
0,1,"{E201, JK}","Jefferson, Thomas",1743,1826,737,{u'text/html': u'http://www.gutenberg.org/eboo...,1,[en],"{United States. Declaration of Independence, U...",The Declaration of Independence of the United ...,Text,1,"[History, Revolution, 1775-1783, Sources, Unit..."
1,2,"{KF, JK}",United States,None,None,212,{u'text/html': u'http://www.gutenberg.org/file...,2,[en],{United States. Constitution. 1st-10th Amendme...,The United States Bill of Rights: The Ten Orig...,Text,1,"[Civil rights, Sources, United States, United ..."
2,3,{E838},"Kennedy, John F. (John Fitzgerald)",1917,1963,34,{u'text/html': u'http://www.gutenberg.org/file...,3,[en],{United States -- Foreign relations -- 1961-19...,John F. Kennedy's Inaugural Address,Text,1,"[1961-1963, Foreign relations, Inaugural addre..."


In [35]:
meta_df['subjects2'][0]

array(['History', 'Revolution, 1775-1783', 'Sources', 'United States',
       'United States. Declaration of Independence'], 
      dtype='|S42')

In [37]:
meta_df['subjects_there'] = meta_df['subjects2'].map(lambda x: 1 if len(x) > 0 else 0)
meta_df[meta_df['subjects_there'] == 0].head(5)

,index,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,language,subjects,title,type,english,subjects2,subjects_there
174,182,{},None,None,None,0,{},182,[en],{},None,Text,1,[],0
175,183,{},None,None,None,0,{},183,[en],{},None,Text,1,[],0
176,184,{},None,None,None,0,{},184,[en],{},None,Text,1,[],0
177,185,{},None,None,None,0,{},185,[en],{},None,Text,1,[],0
178,186,{},None,None,None,0,{},186,[en],{},None,Text,1,[],0


In [38]:
meta_df[meta_df['subjects_there'] == 0].shape

(5254, 15)

## I'm going to remove these columns where there is no author or title

There are many books that showed up in my dataframe with a book id but no data about them. This means that for some reason that book has been made unavailable on Project Gutenberg for some reason or another.
After removing these books, I will still have some empty subjects, but not as many.

In [39]:
meta_df = meta_df[(meta_df['title'].notnull()) & (meta_df['author'].notnull())]

In [41]:
meta_df[meta_df['subjects_there'] == 0].head(3)

,index,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,language,subjects,title,type,english,subjects2,subjects_there
17760,18458,{PQ},"Colonna, Francesco",None,1527,89,{u'image/jpeg': u'http://www.gutenberg.org/cac...,18459,[en],{},Hypnerotomachia: The Strife of Loue in a Dreame,Text,1,[],0
22067,23099,{AC},"De Morgan, Augustus",1806,1871,32,{u'text/plain; charset=us-ascii': u'http://www...,23100,[en],{},"A Budget of Paradoxes, Volume I",Text,1,[],0
23012,24095,{PQ},"Huysmans, J.-K. (Joris-Karl)",1848,1907,29,{u'text/plain; charset=us-ascii': u'http://www...,24096,[en],{},En Route,Text,1,[],0


In [42]:
meta_df.shape

(41429, 15)

In [43]:
meta_df.reset_index(inplace = True, drop = True)

In [50]:
meta_df.drop(['index', 'english', 'subjects_there'], axis = 1, inplace = True)

In [51]:
meta_df.head(2)

,LCC,author,authoryearofbirth,authoryearofdeath,downloads,formats,id,language,subjects,title,type,subjects2
0,"{E201, JK}","Jefferson, Thomas",1743,1826,737,{u'text/html': u'http://www.gutenberg.org/eboo...,1,[en],"{United States. Declaration of Independence, U...",The Declaration of Independence of the United ...,Text,"[History, Revolution, 1775-1783, Sources, Unit..."
1,"{KF, JK}",United States,None,None,212,{u'text/html': u'http://www.gutenberg.org/file...,2,[en],{United States. Constitution. 1st-10th Amendme...,The United States Bill of Rights: The Ten Orig...,Text,"[Civil rights, Sources, United States, United ..."


In [52]:
meta_df = meta_df.reindex_axis(['id', 'title', 'author', 'LCC', 'downloads', 'subjects', 'subjects2', 'formats', 'authoryearofbirth', 'authoryearofdeath', 'type', 'language'], axis = 1)

In [53]:
meta_df.head(2)

,id,title,author,LCC,downloads,subjects,subjects2,formats,authoryearofbirth,authoryearofdeath,type,language
0,1,The Declaration of Independence of the United ...,"Jefferson, Thomas","{E201, JK}",737,"{United States. Declaration of Independence, U...","[History, Revolution, 1775-1783, Sources, Unit...",{u'text/html': u'http://www.gutenberg.org/eboo...,1743,1826,Text,[en]
1,2,The United States Bill of Rights: The Ten Orig...,United States,"{KF, JK}",212,{United States. Constitution. 1st-10th Amendme...,"[Civil rights, Sources, United States, United ...",{u'text/html': u'http://www.gutenberg.org/file...,None,None,Text,[en]


In [54]:
meta_df.shape

(41429, 12)

## At this point I'm going to save a csv file
I want this dataframe to be available to me in a new notebook so I can do further anaysis.

In [ ]:
meta_df.to_csv('gutenberg_metadata_en.csv', encoding = 'utf8')